In [1]:
import pandas as pd
import time as time
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# This function scrapes at least min_posts from Reddit not including [removed] and [deleted] posts, and whether a post is_self or not

def scrape_reddit(subreddit, min_posts, is_self):
     
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < min_posts:
        print('len =', len(df))
        if is_self == 'Y':
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time()),
                    'is_self': True
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            else:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc'],
                    'is_self': True
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)        
        elif is_self == 'N':
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time())
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            else:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
        else:
            print(f"Enter 'Y' or 'N' for third argument.")
            break
            
        print(f'Status code: {res.status_code}')
        time.sleep(0)
        
    return df

In [4]:
wallstreetbets = scrape_reddit('wallstreetbets', 10_000, 'Y')

len = 0
Status code: 200
len = 58
Status code: 200
len = 116
Status code: 200
len = 175
Status code: 200
len = 240
Status code: 200
len = 303
Status code: 200
len = 367
Status code: 200
len = 429
Status code: 200
len = 472
Status code: 200
len = 530
Status code: 200
len = 584
Status code: 200
len = 653
Status code: 200
len = 714
Status code: 200
len = 778
Status code: 200
len = 845
Status code: 200
len = 901
Status code: 200
len = 955
Status code: 200
len = 1017
Status code: 200
len = 1083
Status code: 200
len = 1138
Status code: 200
len = 1194
Status code: 200
len = 1254
Status code: 200
len = 1310
Status code: 200
len = 1366
Status code: 200
len = 1416
Status code: 200
len = 1459
Status code: 200
len = 1511
Status code: 200
len = 1567
Status code: 200
len = 1619
Status code: 200
len = 1673
Status code: 200
len = 1722
Status code: 200
len = 1769
Status code: 200
len = 1811
Status code: 200
len = 1850
Status code: 200
len = 1905
Status code: 200
len = 1958
Status code: 200
len = 1991
S

In [5]:
wallstreetbets.columns
wallstreetbets.index = range(len(wallstreetbets))
wallstreetbets[['id', 'subreddit', 'title', 'selftext', 'domain']].head()

,id,subreddit,title,selftext,domain
0,jatyt2,wallstreetbets,Apple Event Analysis,"So, everything in the Apple event was quite Su...",self.wallstreetbets
1,jatuti,wallstreetbets,Anyone else having Robinhood problems?,Robinhood keeps just saying logging out.. logg...,self.wallstreetbets
2,jattpd,wallstreetbets,Journey to $1M,"Over the past 2 days, I’ve doubled my money tw...",self.wallstreetbets
3,jatntq,wallstreetbets,Maybe Best Buy stock will rise because of the ...,"While the title says it all, Amazon is having ...",self.wallstreetbets
4,jatn0z,wallstreetbets,Calling in the degenerate ER gamblers,Alright you degens I got a play for you take i...,self.wallstreetbets


In [6]:
wallstreetbets.to_csv('./wallstreetbets.csv')

In [7]:
# This function scrapes comments from Reddit

def scrape_reddit_comments(df):
     
    url = 'https://api.pushshift.io/reddit/comment/search'
    df_comments = pd.DataFrame()
#     comments_scraped = 0
    for i in range(len(df)):
        if len(df_comments) == 0:
            params = {
                'size': 100,
                'link_id': df['id'][i],
                'nest_level': 1
            }

#             print(df['id'][i])
#             print(f'Comment length: {len(df_comments)}')

            res = requests.get(url, params)           
            data = res.json()

            if data['data'] == []:
                print('No comments.')
                pass
            else:
                comments = data['data']  
                df_comments = pd.DataFrame(comments)
                df_comments.drop(df_comments[df_comments['author'] == 'AutoModerator'].index, inplace = True)
        else:
            params = {
                'size': 100,
                'link_id': df['id'][i],
                'nest_level': 1
            }
            res = requests.get(url, params)
            data = res.json()
            comments = data['data']
            df2 = pd.DataFrame(comments)
            df_comments = df_comments.append(df2, ignore_index=True)
            df_comments.drop(df_comments[df_comments['author'] == 'AutoModerator'].index, inplace = True)

        print(df['id'][i])
        print(f'df_comments length: {len(df_comments)}.')
        print(f'Status code: {res.status_code}')
        time.sleep(3)
        
    return df_comments

In [8]:
wallstreetbets_comments = scrape_reddit_comments(wallstreetbets)

No comments.
jatyt2
df_comments length: 0.
Status code: 200
No comments.
jatuti
df_comments length: 0.
Status code: 200
No comments.
jattpd
df_comments length: 0.
Status code: 200
jatntq
df_comments length: 1.
Status code: 200
jatn0z
df_comments length: 2.
Status code: 200
jatitv
df_comments length: 5.
Status code: 200
jatgdf
df_comments length: 8.
Status code: 200
jat364
df_comments length: 38.
Status code: 200
jasv53
df_comments length: 89.
Status code: 200
jask09
df_comments length: 93.
Status code: 200
jasd9i
df_comments length: 114.
Status code: 200
jascan
df_comments length: 133.
Status code: 200
jasbqu
df_comments length: 145.
Status code: 200
jas6rt
df_comments length: 149.
Status code: 200
jas62p
df_comments length: 168.
Status code: 200
jarxmw
df_comments length: 209.
Status code: 200
jarvgb
df_comments length: 248.
Status code: 200
jartzf
df_comments length: 264.
Status code: 200
jarmq0
df_comments length: 336.
Status code: 200
jark69
df_comments length: 355.
Status code: 20

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
wallstreetbets_comments[['link_id', 'body']].head()

NameError: name 'wallstreetbets_comments' is not defined

In [ ]:
wallstreetbets_comments.to_csv('./wallstreetbets_comments.csv')